In [2]:
!pip install imbalanced-learn


In [3]:
import numpy as np
import pickle
import pandas as pd
import h5py
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import EfficientFCParameters
import multiprocessing
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.cluster import KMeans
import h5py
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np

np.random.seed(42)

In [4]:
with h5py.File("train.h5", "r") as f_pd:
    y_data = f_pd['y'][:]  
y_df = pd.Series(y_data)  


x_df = pd.read_pickle("all_features.pkl")
x_df.head()
print("Shape de y_df", y_df.shape)
print("Shape de X_df", x_df.shape)

value_counts = y_df.value_counts()
x_df.head()




Shape de y_df (204,)
Shape de X_df (204, 777)


,valor__variance_larger_than_standard_deviation,valor__has_duplicate_max,valor__has_duplicate_min,valor__has_duplicate,valor__sum_values,valor__abs_energy,valor__mean_abs_change,valor__mean_change,valor__mean_second_derivative_central,valor__median,...,valor__fourier_entropy__bins_5,valor__fourier_entropy__bins_10,valor__fourier_entropy__bins_100,valor__permutation_entropy__dimension_3__tau_1,valor__permutation_entropy__dimension_4__tau_1,valor__permutation_entropy__dimension_5__tau_1,valor__permutation_entropy__dimension_6__tau_1,valor__permutation_entropy__dimension_7__tau_1,valor__query_similarity_count__query_None__threshold_0.0,valor__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,0.0,1.0,-0.734864,33.127876,0.003897,-7.016291e-07,2.248299e-07,-0.000244,...,0.314393,0.418924,0.652842,1.222197,1.779523,2.375477,3.002084,3.596227,0.0,0.232910
1,0.0,0.0,0.0,1.0,0.300720,34.903172,0.004165,-5.418686e-07,-1.194152e-07,-0.001068,...,0.273489,0.370036,0.793862,1.148054,1.645622,2.196140,2.777400,3.365738,0.0,0.266963
2,0.0,0.0,0.0,1.0,-0.477569,16.164088,0.002264,-4.119189e-06,-1.837347e-06,-0.000092,...,0.235155,0.305728,0.574520,1.649418,2.545695,3.470808,4.431796,5.335461,0.0,0.207986
3,0.0,0.0,0.0,1.0,0.007569,6.087861,0.001689,-1.726074e-06,-1.227095e-07,0.000244,...,0.239211,0.339942,0.553027,1.663552,2.610122,3.569896,4.564650,5.506580,0.0,0.154306
4,0.0,0.0,0.0,1.0,-0.562194,28.034003,0.004050,1.992890e-07,-9.882629e-09,0.000809,...,0.367289,0.493589,0.808665,1.479723,2.184056,2.926665,3.690862,4.417537,0.0,0.336108


In [5]:
y_df.head()

0    0
1    1
2    0
3    0
4    0
dtype: int64

In [6]:
class SVM:
    def __init__(self, kernel_type='linear', regularization_parameter=10000.0, max_iterations=500, polynomial_degree=3, gauss_gamma=1,sigmoid_alpha=0.001, sigmoid_c=0):

        self.kernel_type = kernel_type
        self.polynomial_degree = polynomial_degree
        self.gauss_gamma = gauss_gamma
        self.regularization_parameter = regularization_parameter
        self.max_iterations = max_iterations
        self.support_vector_indices = None
        #Kernell sgimoideo
        self.sigmoid_alpha = sigmoid_alpha
        self.sigmoid_c = sigmoid_c

    def _compute_kernel_matrix(self, X1, X2):

        # selección del kernel
        if self.kernel_type == 'linear':
            return np.dot(X1, X2.T)
        elif self.kernel_type == 'poly':
            return np.dot(X1, X2.T)**self.polynomial_degree
        elif self.kernel_type == 'gauss':
            if X1 is X2:
                pairwise_distances = np.sum((X1[:, np.newaxis] - X2)**2, axis=2)
            else:
                pairwise_distances = np.sum((X2 - X1[:, np.newaxis])**2, axis=2)
            return np.exp(-self.gauss_gamma * pairwise_distances)
        elif self.kernel_type == 'sigmoid':  # Caso para el kernel sigmoideo
          return np.tanh(self.sigmoid_alpha * np.dot(X1, X2.T) + self.sigmoid_c)
        else:
            raise ValueError(f"Kernel no soportado: {self.kernel_type}")

    def fit(self, feature_matrix, target_labels):
        self.features = feature_matrix.copy()
        self.labels = target_labels * 2 - 1  #etiquetas [0, 1] a [-1, 1]
        self.lagrange_multipliers = np.zeros_like(self.labels, dtype=float) # Estos son números que va a aprender para decidir qué puntos son vectores de soporte.

        kernel_values = self._compute_kernel_matrix(self.features, self.features) # k(x(i), x(j))


        # el "self.labels" y "self.labels[:, np.newaxis]" representa las clases reales de los datos de entrenamiento
        self.kernel_matrix = kernel_values * self.labels[:, np.newaxis] * self.labels #t(i)*t(j)*k(x(i)T, x(j))

        # optimización // para encontrar los multiplicadores de Lagrange
        for _ in range(self.max_iterations):
            for primary_index in range(len(self.lagrange_multipliers)):
                secondary_index = np.random.randint(0, len(self.lagrange_multipliers))

                quadratic_term = self.kernel_matrix[[[primary_index, primary_index], [secondary_index, secondary_index]],
                                                     [[primary_index, secondary_index], [primary_index, secondary_index]]]

                current_multipliers = self.lagrange_multipliers[[primary_index, secondary_index]]

                error_terms = 1 - np.sum(self.lagrange_multipliers * self.kernel_matrix[[primary_index, secondary_index]], axis=1)

                direction_vector = np.array([-self.labels[secondary_index], self.labels[primary_index]])

                optimal_step = np.dot(error_terms, direction_vector) / (np.dot(np.dot(quadratic_term, direction_vector), direction_vector) + 1E-15)

                constrained_step = self._restrict_to_valid_range(optimal_step, current_multipliers, direction_vector)

                self.lagrange_multipliers[[primary_index, secondary_index]] = current_multipliers + direction_vector * constrained_step #multiplicadores de lagrange optimizados

        # identificamos los vectores de soporte y calcular el sesgo

        self.support_vector_indices = np.nonzero(self.lagrange_multipliers > 1E-15)[0] # Identifica qué puntos de entrenamiento son vectores de soporte.

        if len(self.support_vector_indices) > 0: # calculo de sedsgo
            self.b = np.sum((1.0 - np.sum(self.kernel_matrix[self.support_vector_indices] *
                                            self.lagrange_multipliers, axis=1)) *
                               self.labels[self.support_vector_indices]) / len(self.support_vector_indices)
        else:
            self.b = 0.0

        if self.kernel_type == 'linear': # Cálculo del vector de pesos W (para kernel lineales)
            self.W = np.sum((self.lagrange_multipliers * self.labels)[:, np.newaxis] * self.features, axis=0)
        return self


    def predict(self, X):
        #valores aplicando el kernel
        kernel_values = self._compute_kernel_matrix(X, self.features)

        #predicciones [1, -1]
        decision_values = np.sum(kernel_values * self.labels * self.lagrange_multipliers, axis=1) + self.b
        return (np.sign(decision_values) + 1) // 2  # conversion a [0,1]

    def _restrict_to_valid_range(self, step_size, current_values, direction_vector):
        #restringimos el paso de optimización para mantener los multiplicadores dentro de los límites permitidos

        step_size = (np.clip(current_values + step_size*direction_vector, 0, self.regularization_parameter) -
                    current_values)[1]/direction_vector[1]
        return (np.clip(current_values + step_size*direction_vector, 0, self.regularization_parameter) -
               current_values)[0]/direction_vector[0]

    def get_support_vectors(self):

        if self.support_vector_indices is None:
            raise ValueError("El modelo debe ser entrenado antes de obtener los vectores de soporte")

        return self.support_vector_indices



In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(x_df.to_numpy())
y = y_df.to_numpy()


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Separar en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

#smote = SMOTE(random_state=42)
#smote.fit_resample(X_train, y_train)
X_train_resampled, y_train_resampled = X_train, y_train

svm_gauss = SVM(kernel_type='gauss', regularization_parameter=1.0, gauss_gamma=0.1)
svm_gauss.fit(X_train_resampled, y_train_resampled)

y_pred_gauss_custom = svm_gauss.predict(X_test)
accuracy_gauss_custom = accuracy_score(y_test, y_pred_gauss_custom)

print("gauss Kernel (Custom SVM):")
print(classification_report(y_test, y_pred_gauss_custom))

gauss Kernel (Custom SVM):
              precision    recall  f1-score   support

           0       0.74      1.00      0.85        46
           1       0.00      0.00      0.00        16

    accuracy                           0.74        62
   macro avg       0.37      0.50      0.43        62
weighted avg       0.55      0.74      0.63        62



/home/marce/Documentos/Ciclo6/Machine Learning/Proyecto1/project-1-classification-2025-1-b/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/marce/Documentos/Ciclo6/Machine Learning/Proyecto1/project-1-classification-2025-1-b/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/marce/Documentos/Ciclo6/Machine Learning/Proyecto1/project-1-classification-2025-1-b/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precis

In [9]:

svm_sigmoid = SVM(kernel_type='sigmoid', regularization_parameter=1.0, gauss_gamma=0.1)
svm_sigmoid.fit(X_train, y_train)
y_pred_sigmoid_custom = svm_sigmoid.predict(X_test)

print("rbf Kernel (Custom SVM):")
print(classification_report(y_test, y_pred_sigmoid_custom))

rbf Kernel (Custom SVM):
              precision    recall  f1-score   support

           0       0.77      0.96      0.85        46
           1       0.60      0.19      0.29        16

    accuracy                           0.76        62
   macro avg       0.69      0.57      0.57        62
weighted avg       0.73      0.76      0.71        62



In [10]:

svm_sigmoid = SVM(kernel_type='sigmoid', regularization_parameter=1.0,
                  sigmoid_alpha=0.01, sigmoid_c=0)

svm_sigmoid.fit(X_train_resampled, y_train_resampled)
y_pred_sigmoid_custom = svm_sigmoid.predict(X_test)

print("rbf Kernel (Custom SVM):")
print(classification_report(y_test, y_pred_sigmoid_custom))

# Guardar el mejor modelo en un archivo .pkl
with open("simple_svm_sigmoid.pkl", "wb") as f:
    pickle.dump(svm_sigmoid, f)

print("\ modelo guardado como 'simple_svm_sigmoid.pkl'")

rbf Kernel (Custom SVM):
              precision    recall  f1-score   support

           0       0.82      0.67      0.74        46
           1       0.38      0.56      0.45        16

    accuracy                           0.65        62
   macro avg       0.60      0.62      0.59        62
weighted avg       0.70      0.65      0.66        62

\ modelo guardado como 'simple_svm_sigmoid.pkl'


In [11]:
for alpha in [0.001, 0.01, 0.1]:
    for c in [-1, 0, 1]:
        svm_sigmoid = SVM(kernel_type='sigmoid', regularization_parameter=1.0,
                          sigmoid_alpha=alpha, sigmoid_c=c)
        svm_sigmoid.fit(X_train_resampled, y_train_resampled)
        preds = svm_sigmoid.predict(X_test)
        print(f"alpha={alpha}, c={c}")
        print(classification_report(y_test, preds))


alpha=0.001, c=-1
              precision    recall  f1-score   support

           0       0.82      0.78      0.80        46
           1       0.44      0.50      0.47        16

    accuracy                           0.71        62
   macro avg       0.63      0.64      0.64        62
weighted avg       0.72      0.71      0.71        62

alpha=0.001, c=0
              precision    recall  f1-score   support

           0       0.78      0.83      0.80        46
           1       0.38      0.31      0.34        16

    accuracy                           0.69        62
   macro avg       0.58      0.57      0.57        62
weighted avg       0.67      0.69      0.68        62

alpha=0.001, c=1
              precision    recall  f1-score   support

           0       0.79      0.83      0.81        46
           1       0.43      0.38      0.40        16

    accuracy                           0.71        62
   macro avg       0.61      0.60      0.60        62
weighted avg       0.7

In [12]:

best_f1 = 0
best_model = None
best_params = None

for alpha in [0.001, 0.01, 0.1]:
    for c in [-1, 0, 1]:
        print(f"Entrenando con alpha={alpha}, c={c}...")
        svm_sigmoid = SVM(kernel_type='sigmoid', regularization_parameter=1.0,
                          sigmoid_alpha=alpha, sigmoid_c=c)
        svm_sigmoid.fit(X_train_resampled, y_train_resampled)
        preds = svm_sigmoid.predict(X_test)

        current_f1 = f1_score(y_test, preds, average='macro')
        print(f"F1-macro: {current_f1:.4f}")
        print(classification_report(y_test, preds))

        if current_f1 > best_f1:
            best_f1 = current_f1
            best_model = svm_sigmoid
            best_params = (alpha, c)

# Guardar el mejor modelo en un archivo .pkl
with open("mejor_modelo_svm_sigmoid.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("\nMejor modelo guardado como 'mejor_modelo_svm_sigmoid.pkl'")
print(f"Mejor combinación: alpha={best_params[0]}, c={best_params[1]}, F1-macro={best_f1:.4f}")


Entrenando con alpha=0.001, c=-1...
F1-macro: 0.6077
              precision    recall  f1-score   support

           0       0.80      0.78      0.79        46
           1       0.41      0.44      0.42        16

    accuracy                           0.69        62
   macro avg       0.61      0.61      0.61        62
weighted avg       0.70      0.69      0.70        62

Entrenando con alpha=0.001, c=0...
F1-macro: 0.5724
              precision    recall  f1-score   support

           0       0.78      0.83      0.80        46
           1       0.38      0.31      0.34        16

    accuracy                           0.69        62
   macro avg       0.58      0.57      0.57        62
weighted avg       0.67      0.69      0.68        62

Entrenando con alpha=0.001, c=1...
F1-macro: 0.6043
              precision    recall  f1-score   support

           0       0.79      0.83      0.81        46
           1       0.43      0.38      0.40        16

    accuracy             